In [2]:
#Создаем соединение с SQL (+создаем ввод пароля)
import psycopg2
import getpass
import pandas as pd
import numpy as np
import datetime

conn = psycopg2.connect(user='postgres', 
                        database='postgres', 
                        host='localhost', 
                        port='5433', 
                        password = getpass.getpass('Insert password: ')
                       )
print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5433', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [3]:
#Получение данных
#Нам необходимы следующие данные: ISIN, тип купона (постоянный и дисконтный), столбец, указывающий, является ли 
#облигация амортизационной, купон, количество купонов в год, цена и дата обратного выкупа по оферете,
#кколичество выплаченных купонов, номинальная стоимость, дата погашения облигации, дата выпуска облигации (как дата начала торгов)
#Из всего пречисленного нас особо интересуют непогашенные облигации, также убираем отсутствующие значения ISIN
#Поскольку по облигации может быть разное значение купона, отбираем только максимальное его значение, также берем 
#максимальное значение buybackdate и buybackprice для отбора только одного для каждой облигации
import pandas as pd
query = """SELECT "ISIN", "CouponType", "AmortisedMty", "CouponPerYear", "NumCoupons",
            "CPN", "BUYBACKDATE", "BUYBACKPRICE", "FaceValue", "EndMtyDate", "CPN_PERIOD", "FirstDate"
            FROM (
				SELECT DISTINCT public.bond_description."ISIN, RegCode, NRDCode", public.bond_description."CouponType", 
				public.bond_description."AmortisedMty", public.bond_description."CouponPerYear", public.bond_description."NumCoupons",
				public.bond_description."FaceValue", public.bond_description."EndMtyDate" 
				FROM public.bond_description INNER JOIN public.quotes
				ON public.quotes."ISIN" = public.bond_description."ISIN, RegCode, NRDCode"  
				WHERE "ISIN, RegCode, NRDCode" IS NOT NULL AND "Status" != 'Погашен') AS S1
			INNER JOIN (SELECT "ISIN", MAX("CPN_PERIOD") AS "CPN_PERIOD", MAX("CPN") AS "CPN", MAX("BUYBACKDATE") AS "BUYBACKDATE", MAX("BUYBACKPRICE") AS "BUYBACKPRICE", MIN("TIME") AS "FirstDate" 
                FROM public.quotes GROUP BY "ISIN") AS S2
			ON S1."ISIN, RegCode, NRDCode"=S2."ISIN";"""
df = pd.read_sql_query(query, conn) 
df['EndMtyDate'] = pd.to_datetime(df['EndMtyDate'])
df.BUYBACKPRICE = df.BUYBACKPRICE.fillna(0) 
df

,ISIN,CouponType,AmortisedMty,CouponPerYear,NumCoupons,CPN,BUYBACKDATE,BUYBACKPRICE,FaceValue,EndMtyDate,CPN_PERIOD,FirstDate
0,RU000A0JTP58,Переменный,False,2,6,69.8100,None,0.0,1000.0,2016-02-23,182.0,2013-02-27
1,RU000A0JRN29,Переменный,False,2,6,49.9100,None,0.0,1000.0,2014-07-28,184.0,2011-07-29
2,RU000A0JUR04,Переменный,False,2,20,49.8630,9999-05-05,100.0,1000.0,2024-07-05,182.0,2014-07-21
3,RU000A0JU9V1,Постоянный,False,2,11,34.6900,None,0.0,1000.0,2019-05-15,189.0,2013-11-28
4,RU000A0JV1Y1,Переменный,False,2,20,59.3400,9999-05-04,100.0,1000.0,2024-11-28,182.0,2014-12-11
5,RU000A0JUEH0,Переменный,False,2,14,49.8630,9999-05-05,100.0,1000.0,2021-01-19,182.0,2014-02-05
6,RU000A0JT8R4,Переменный,False,2,14,59.8356,9999-05-04,100.0,1000.0,2019-10-18,182.0,2012-12-10
7,RU000A0JV1W5,Переменный,False,2,20,59.3400,9999-05-04,100.0,1000.0,2024-11-28,182.0,2014-12-11
8,RU000A0JTYM0,Переменный,False,2,20,39.6411,9999-05-04,100.0,1000.0,2023-05-30,182.0,2013-07-15
9,RU000A0JUQQ5,Переменный,False,2,10,61.7534,9999-05-05,100.0,1000.0,2019-07-10,184.0,2014-07-11


In [4]:
#Используем следуюшие аргументы: ISIN, дата, логин
def my_fun(ISIN, RegCode, NRDCode, date, db_login):
    conn=psycopg2.connect(user=db_login['user'],
                      database=db_login['database'],
                      host=db_login['host'],
                      port=db_login['port'],
                      password=getpass.getpass('Insert password: '))

In [ ]:
#Здесь приведена попытка написать код для неамортизационных облигаций с постоянным купоном. В какой-то момент Python 
#перестал работать именно на этой ячейке(пробовала перезапускать все) 
#подозреваю, что полученные значения нужно еще сгруппировать по ISIN и использовать pivot для вывода таблицы


import pandas as pd
import numpy as np
import datetime

df['EndMtyDate'] = pd.to_datetime(df['EndMtyDate'])
ISIN = df['ISIN']
coup = df['CPN']
coup_per_year = df['CouponPerYear']
coup_num = df['NumCoupons']
df['FirstDate'] = pd.to_datetime(df['FirstDate'])
buyback_date = df['BUYBACKDATE']
date = df['FirstDate']
buyback_price = df['BUYBACKPRICE']
date = pd.to_datetime(date)
AmortisedMty = df.AmortisedMty.astype(str)
CouponType = df.CouponType.astype(str)
payment = coup
df.CPN_PERIOD = pd.to_timedelta(df.CPN_PERIOD)
for i in range (0, len(df)): 
    if df.AmortisedMty.values[i] == False and df.CouponType.values[i] == 'Постоянный':
        if df.BUYBACKDATE.values[i] is None:
            while date.values[i] < df.EndMtyDate.values[i]:
                payment.values[i] = coup.values[i]
                date.values[i] = date.values[i] + df.CPN_PERIOD.values[i]
            if date.values[i] == df.EndMtyDate.values[i]:
                payment.values[i] = df.CPN.values[i] + df.FaceValue.values[i]
                date.values[i] = df.EndMtyDate.values[i] 
        if df.BUYBACKDATE.values[i] is not None:
            while date.values[i] < df.BUYBACKDATE.values[i]:
                payment.values[i] = df.CPN.values[i]
                date.values[i] = date.values[i] + df.CPN_PERIOD.values[i]
            if date.values[i] == df.BUYBACKDATE.values[i]:
                payment.values[i] = df.CPN.values[i] + df.BUYBACKPRICE.values[i]
                date.values[i] = df.BUYBACKDATE.values[i] 
            
#if df.AmortisedMty.values[i] == True and df.CouponType.values[i] == 'Постоянный':
      # if df.BUYBACKDATE.values[i] is None:
        
    
      # if df.BUYBACKDATE.values[i] is not None:    
            

#if df.CouponType.values[i] == 'Дисконт':
     # if df.BUYBACKDATE.values[i] is None:
        
    
     # if df.BUYBACKDATE.values[i] is not None:    